In [1]:
import numpy as np
import tensorflow as tf
from itertools import product
import random
#import tfnumpy as tfnp

In [2]:
x1 = np.array([[1, 0], [0, 1], [2, -1.4]], dtype=np.float32)
x2 = np.array([[0, -1], [1, 0], [-1.5, 0.6]], dtype=np.float32)
x = np.zeros((3, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

# Using numpy first

In [3]:
a = np.random.uniform(size=x.shape[2])
b = np.random.uniform(size=x.shape[2])

In [4]:
x[:, :, 0]*a[0]+x[:, :, 1]*a[1]

array([[ 0.11478091, -0.70786967],
       [ 0.70786967,  0.11478091],
       [-0.83224269,  0.26402855]])

In [5]:
Ta = sum(x[:, :, i]*a[i] for i in range(x.shape[2]))
Ta

array([[ 0.11478091, -0.70786967],
       [ 0.70786967,  0.11478091],
       [-0.83224269,  0.26402855]])

In [6]:
Tb = sum(x[:, :, i]*b[i] for i in range(x.shape[2]))
Tb

array([[ 0.51066343, -0.74293773],
       [ 0.74293773,  0.51066343],
       [-0.09307973, -0.26916614]])

In [7]:
eigvals_u, eigvecs_u = np.linalg.eig(np.matmul(Ta, np.linalg.pinv(Tb)))

In [18]:
eigvals_v, eigvecs_v = np.linalg.eig(np.transpose(np.matmul(np.linalg.pinv(Ta), Tb)))

In [19]:
eigvals_u

array([6.90145247e-01+0.44574518j, 6.90145247e-01-0.44574518j,
       7.40413414e-17+0.j        ])

In [20]:
eigvals_v

array([0.79975795+0.3275891j, 0.79975795-0.3275891j])

In [25]:
np.matmul(Tb, np.linalg.pinv(Tb))

array([[ 0.96801155,  0.04335439,  0.17054498],
       [ 0.04335439,  0.94124119, -0.23114196],
       [ 0.17054498, -0.23114196,  0.09074726]])

In [12]:
# pair up eigenvalues of Ta and Tb
idx_pairs = []
tol = 1e-5

for i, eigval_u in enumerate(eigvals_u):
    for j, eigval_v in enumerate(eigvals_v):
        if abs(eigval_u - 1/eigval_v) < tol:
            idx_pairs += [(i, j)]
            break

idx_pairs

[]

In [14]:
nbcomp = len(idx_pairs)
A = np.zeros((nbcomp*x.shape[2], nbcomp*x.shape[2]))
B = np.zeros(nbcomp*x.shape[2])
eqidx = 0
for k in range(x.shape[2]):
    for i, j in random.choices(list(product(range(x.shape[0]), range(x.shape[1]))), k=nbcomp):
        #print("{} {} {}".format(i, j, k))
        B[eqidx] = x[i, j, k]
        for ck in range(nbcomp):
            A[eqidx, ck*nbcomp+k] = eigvecs_u[idx_pairs[ck][0], i]*eigvecs_v[idx_pairs[ck][1], j]
        eqidx += 1

In [17]:
sol = np.linalg.solve(A, B)
sol

array([ 5.46398458, -1.25720609, -3.21753026,  4.2376251 ])

In [20]:
eigvecs_w = sol.reshape((nbcomp, x.shape[2]), order='F')
eigvecs_w

array([[ 5.46398458, -3.21753026],
       [-1.25720609,  4.2376251 ]])

In [34]:
# verfication
xh = np.zeros(x.shape)
for i, j, k in product(range(x.shape[0]), range(x.shape[1]), range(x.shape[2])):
    xh[i, j, k] = sum([eigvecs_u[alpha, i]*eigvecs_v[alpha, j]*eigvecs_w[alpha, k] for alpha in range(nbcomp)])

In [35]:
xh

array([[[ 0.19853161, -1.5467626 ],
        [-0.73690431,  2.76013739]],

       [[ 2.38614556, -0.03303901],
        [-0.10405237, -2.17092161]],

       [[ 1.41715524, -1.87432896],
        [-0.93658567,  2.24318313]]])

In [36]:
x

array([[[ 1.        ,  0.        ],
        [ 0.        , -1.        ]],

       [[ 0.        ,  1.        ],
        [ 1.        ,  0.        ]],

       [[ 2.        , -1.5       ],
        [-1.39999998,  0.60000002]]])